
# Imports

In [1]:
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive')
#!git clone  https://github.com/matterport/Mask_RCNN.git
# Copy train dataset and matterport Mask_RCNN
!cp -r /content/drive/MyDrive/Mask_RCNN /content/Mask_RCNN/
!cp -r /content/drive/MyDrive/Bc-proj/train_data /content/train_data

!mkdir /content/logs
%cd /content/Mask_RCNN
!pip install -r requirements.txt
!pip install imgaug
!pip install h5py==2.10.0 
!python setup.py install





TensorFlow 1.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/Mask_RCNN
     |████████████████████████████████| 163kB 5.7MB/s 
     |████████████████████████████████| 204kB 27.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=ac0b3ddf6346d28b67721b275fd85d246cda9ebcc24479cead735bd2546655db
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.2 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
     |████████████████████████████████| 2.9MB 4.9MB/s 
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
/usr/local/lib/python3.7/dist-packages/setuptoo

In [2]:
import os


import cv2 as cv
import sys
import random
import math
import json
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
import imgaug

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn.model import log
from mrcnn.utils import Dataset
# Copy weights  pretrainned on coco dataset
!cp /content/drive/MyDrive/Bc-proj/mask_rcnn_coco.h5 ..






Using TensorFlow backend.


Config

In [3]:

ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.abspath("/content/logs")
# Local path to trained weights file


COCO_MODEL_PATH = os.path.join('..', "mask_rcnn_coco.h5")

class McConfig(Config):
    # Give the configuration a recognizable name
    NAME = "Mc"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 bone

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 600
    IMAGE_MAX_DIM = 1600

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    #Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 6

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 200

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 15


config = McConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1600
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  600
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1600 1600    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.00075
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE       

dataset

In [4]:
class McDataset(utils.Dataset):
    """Loads metacarpal bones dataset and anotated data.
    """

    def load_mc(self, anot_dir):
        # Add classes and images to dataset
        self.add_class("Mc", 1, "mc bone")
        if not os.path.isdir(anot_dir):
              raise ValueError(f'{image_path} is not a valit file.')
        anot_files = []
        for path, dirs, files in os.walk(anot_dir):
            anot_files += [os.path.join(path, f) for f in files]
        #print(len(anot_files))
        for f in anot_files:
            with open(f) as handle:
                file = json.load(handle)
            polygons = []
            for shape in file['shapes']:
                if shape['label'] == 'mc':
                    polygons = [[int(x), int(y)] for x, y in shape['points']]
            image_path = os.path.join(anot_dir + "/../", file['imagePath'].split('../')[-1])
            if not os.path.isfile(image_path):
              raise ValueError(f'{image_path} is not a valit file.')
            height, width = file['imageHeight'], file['imageWidth']
            self.add_image(
                "Mc",
                image_id=image_path.split('/')[-1],
                path=image_path,
                width=width,
                height=height,
                polygons=polygons
            )

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], 3], dtype=np.uint8)
        cv.fillPoly(mask, [np.array(info["polygons"], dtype=np.int32)], 1)
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
            info = self.image_info[image_id]
            print(info)
            return info['path']
# image augmentations
augmentations=imgaug.augmenters.Sometimes(1/3,imgaug.augmenters.OneOf(
                                            [
                                            imgaug.augmenters.AdditiveGaussianNoise(scale=(0, 0.08*255)), 
                                            imgaug.augmenters.GaussianBlur(sigma=(0.0, 3.0)), 
                                            imgaug.augmenters.GammaContrast((0.5, 2.0)), 
                                            imgaug.augmenters.LogContrast(gain=(0.6, 1.4)), 
                                            imgaug.augmenters.PiecewiseAffine(scale=(0.01, 0.05)),
                                            imgaug.augmenters.PerspectiveTransform(scale=(0.01, 0.15)),
                                            imgaug.augmenters.AverageBlur(k=(2, 8))
                                            ]
                                                           )
                                             )
def train(model, dataset='/data/public/Bakalářka/dataset/train_data/annotation/', n_epochs=200):
    dataset_train = McDataset()
    dataset_train.load_mc(dataset + '/train_data/annotation')
    dataset_train.prepare()
    dataset_val = McDataset()
    dataset_val.load_mc(dataset + '/val_data/annotation')
    dataset_val.prepare()
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=n_epochs,
                layers='heads',
                augmentation=augmentations
                )
         
    

train


In [5]:
config = McConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=MODEL_DIR)
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc",
        "mrcnn_bbox", "mrcnn_mask"])
train(model,'/content/train_data/')
%load_ext tensorboard
%tensorboard --logdir /content/logs

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead

Starting at epoch 0. LR=0.00075

Checkpoint Path: /content/logs/mc20210719T0843/mask_rcnn_mc_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/tensorflow-1.15.2/python3.7/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/100
170/170 [==============================] - 194s 1s/step - loss: 1.3051 - val_loss: 0.6044

Epoch 2/100
170/170 [==============================] - 151s 886ms/step - loss: 0.6716 - val_loss: 0.6479
Epoch 3/100
170/170 [==============================] - 149s 878ms/step - loss: 0.5093 - val_loss: 0.3859
Epoch 4/100
170/170 [==============================] - 149s 874ms/step - loss: 0.5287 - val_loss: 0.3795
Epoch 5/100
170/170 [==============================] - 148s 872ms/step - loss: 0.4833 - val_loss: 0.7131
Epoch 6/100
170/170 [==============================] - 145s 853ms/step - loss: 0.4615 - val_loss: 0.4322
Epoch 7/100
170/170 [==============================] - 145s 853ms/step - loss: 0.4623 - val_loss: 0.3243
Epoch 8/100
170/170 [==============================] - 148s 873ms/step - loss: 0.4136 - val_loss: 0.2019
Epoch 9/100
170/170 [==============================] - 147s 865ms/step - loss: 0.3788 - val_loss: 0.2292
Epoch 10/100
170/170 [==============================] - 1